In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# get the model and video
# video:
# https://www.pexels.com/video/people-using-escalators-in-the-mall-1338592/

# v4 tiny weights:
# https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
# v4 tiny config:
# https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg7

# v3 weights:
# https://pjreddie.com/media/files/yolov3.weights
# v3 config:
# https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg

# labels:
# https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
# and place it to appropriate location

In [2]:
# define a path to folder with the video and yolo files
path = 'drive/MyDrive/Colab_Notebooks_2/yolo_video_inference' 

In [3]:
# Detecting Objects on Image with OpenCV deep learning library
#
# How does YOLO-v3 Algorithm works for this example case:
# STEP1: Reading input video
# STEP2: Loading YOLO v3 Network
# STEP3: Reading frames in the loop
# STEP4: Getting blob from the frame
# STEP4: Implementing Forward Pass
# STEP5: Getting Bounding Boxes
# STEP6: Non-maximum Suppression
# STEP7: Drawing Bounding Boxes with Labels
# STEP8: creating a new video by writing processed frames
#
# Result:
# New video file with Detected Objects, Bounding Boxes and Labels


# Importing needed libraries
import os
import re
import numpy as np
import cv2
import time
print (cv2.__version__)


4.7.0


In [4]:

"""
==================     STEP1   ===================
Start of: Reading input video
"""
#NOTE:
# Defining 'VideoCapture' object and reading video from a file make sure that the path and file name is correct
video = cv2.VideoCapture(os.path.join(path, 'pexels-richarles-moral-1338592-1920x1080-30fps.mp4'))

# Preparing variable for writer that we will use to write processed frames
writer = None

# Preparing variables for spatial dimensions of the frames
h, w = None, None

"""
End of:
Reading input video
"""


'\nEnd of:\nReading input video\n'

In [41]:
####### YOLO V4 TINY #######

"""
==================     STEP2  ===================
Start of: Loading YOLO v4 network
"""

# Loading COCO class labels from file and Opening file
with open(os.path.join(path, 'coco.names')) as f:
    # Getting labels reading every line
    # and putting them into the list
    labels = [line.strip() for line in f]


# # Check point
# print('List with labels names:')
# print(labels)

# Loading trained YOLO v3 Objects Detector with the help of 'dnn' library from OpenCV
network = cv2.dnn.readNetFromDarknet(os.path.join(path, 'yolov4-tiny.cfg'),
                                     os.path.join(path, 'yolov4-tiny.weights'))

# Getting list with names of all layers from YOLO v3 network
layers_names_all = network.getLayerNames()

# # Check point
# print()
# print(layers_names_all)

# Getting only output layers' names that we need from YOLO v3 algorithm
# with function that returns indexes of layers with unconnected outputs
#layers_names_output = [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()] # change needed for yolov4tiny
layers_names_output = [layers_names_all[i - 1] for i in network.getUnconnectedOutLayers()]


# # Check point
# print()
# print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']

# Setting minimum probability to eliminate weak predictions
probability_minimum = 0.5

# Setting threshold for filtering weak bounding boxes
# with non-maximum suppression
threshold = 0.3

# Generating colours for representing every detected object
# with function randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# # Check point
# print(colours.shape)
# print(colours[0])

"""
End of:
Loading YOLO v3 network
"""

'\nEnd of:\nLoading YOLO v3 network\n'

In [5]:
####### YOLO V3 #######

"""
==================     STEP2  ===================
Start of: Loading YOLO v3 network
"""

# Loading COCO class labels from file and Opening file
with open(os.path.join(path, 'coco.names')) as f:
    # Getting labels reading every line
    # and putting them into the list
    labels = [line.strip() for line in f]


# # Check point
# print('List with labels names:')
# print(labels)

# Loading trained YOLO v3 Objects Detector with the help of 'dnn' library from OpenCV
network = cv2.dnn.readNetFromDarknet(os.path.join(path, 'yolov3.cfg'),
                                     os.path.join(path, 'yolov3.weights'))

# Getting list with names of all layers from YOLO v3 network
layers_names_all = network.getLayerNames()

# # Check point
# print()
# print(layers_names_all)

# Getting only output layers' names that we need from YOLO v3 algorithm
# with function that returns indexes of layers with unconnected outputs
#layers_names_output = [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()] # change needed for yolov4tiny
layers_names_output = [layers_names_all[i - 1] for i in network.getUnconnectedOutLayers()]


# # Check point
# print()
#print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']

# Setting minimum probability to eliminate weak predictions
probability_minimum = 0.5

# Setting threshold for filtering weak bounding boxes
# with non-maximum suppression
threshold = 0.3

# Generating colours for representing every detected object
# with function randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# # Check point
# print(colours.shape)
# print(colours[0])

"""
End of:
Loading YOLO v3 network
"""

'\nEnd of:\nLoading YOLO v3 network\n'

In [6]:



"""
==================     STEP3  ===================
Start of: Reading frames in the loop
"""

# Defining variable for counting frames at the end we will show total amount of processed frames
f = 0

# Defining variable for counting total time At the end we will show time spent for processing all frames
t = 0

# Defining loop for catching frames 
while True:
    # Capturing frame-by-frame
    ret, frame = video.read()

    # If the frame was not retrieved e.g.: at the end of the video, then we break the loop
    if not ret:
        break

    # Getting spatial dimensions of the frame as we do it only once from the very beginning
    # all other frames have the same dimension
    if w is None or h is None:
        # Slicing from tuple only first two elements
        h, w = frame.shape[:2]
    
    """
    End of: Reading frame in loop
    """

    """
    ==================     STEP4  ===================
    Start of: Getting blob from current frame
    """

    # Getting blob from current frame
    # The 'cv2.dnn.blobFromImage' function returns 4-dimensional blob from current
    # frame after mean subtraction, normalizing, and RB channels swapping
    # Resulted shape has number of frames, number of channels, width and height
    # e.g.:
    # blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)

    """
    End of: Getting blob from current frame
    """

    """
    ==================     STEP4  ===================
    Start of: Implementing Forward pass
    """

    # Implementing forward pass with our blob and only through output layers
    # Calculating at the same time, needed time for forward pass
    network.setInput(blob)  # setting blob as input to the network
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increasing counters for frames and total time
    f += 1
    t += end - start

    # Showing spent time for single current frame
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))
    
    """
    End of:Implementing Forward pass
    """
    
    """
    ==================     STEP5  ===================
    Start of: Getting bounding boxes
    """

    # Preparing lists for detected bounding boxes, obtained confidences and class's number
    bounding_boxes = []
    confidences = []
    classIDs = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 80 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]
            
            # # Every 'detected_objects' numpy array has first 4 numbers with
            # # bounding box coordinates and rest 80 with probabilities
            #  # for every class
            # print(detected_objects.shape)  # (85,)

            # Eliminating weak predictions with minimum probability
            if confidence_current > probability_minimum:
                # Scaling bounding box coordinates to the initial frame size
                # YOLO data format keeps coordinates for center of bounding box
                # and its current width and height
                # That is why we can just multiply them elementwise
                # to the width and height
                # of the original frame and in this way get coordinates for center
                # of bounding box, its width and height for original frame
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Now, from YOLO data format, we can get top left corner coordinates that are x_min and y_min
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                classIDs.append(class_current)
            	
        """	
        End of: Getting bounding boxes
        """

    """
    ==================     STEP6   ===================
    Start of: Non-maximum suppression
    """

    # Implementing non-maximum suppression of given bounding boxes
    # With this technique we exclude some of bounding boxes if their
    # corresponding confidences are low or there is another
    # bounding box for this region with higher confidence

    # It is needed to make sure that data type of the boxes is 'int'
    # and data type of the confidences is 'float'
    # https://github.com/opencv/opencv/issues/12789
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                               probability_minimum, threshold)
    
    """
    End of: Non-maximum suppression
    """

    """
    ==================     STEP6   ===================
    Start of: Drawing bounding boxes and labels
    """

    # Checking if there is at least one detected object after non-maximum suppression
    if len(results) > 0:
        # Going through indexes of results
        for i in results.flatten():
            # Getting current bounding box coordinates, its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            # Preparing colour for current bounding box and converting from numpy array to list
            colour_box_current = colours[classIDs[i]].tolist()

            # print(type(colour_box_current))  # <class 'list'>
            # print(colour_box_current)  # [172 , 10, 127]

            # Drawing bounding box on the original current frame
            cv2.rectangle(frame, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)

            # Preparing text with label and confidence for current bounding box
            text_box_current = '{}: {:.4f}'.format(labels[int(classIDs[i])],
                                                   confidences[i])

            # Putting text with label and confidence on the original image
            cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)

    """
    End of:
    Drawing bounding boxes and labels
    """
    
    
    """
    ==================     STEP7   ===================
    Start of: Writing processed frame into the file
    """

    # Initializing writer
    # we do it only once from the very beginning when we get spatial dimensions of the frames
    if writer is None:
        # Constructing code of the codec to be used in the function VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        writer = cv2.VideoWriter(os.path.join(path, 'video_result.mp4'), fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    # Write processed current frame to the file
    writer.write(frame)

"""
End of: Writing processed frame into the file
"""

"""
End of: Writing processed frame into the file
"""


# Printing final results
print()
print('Total number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))
print('FPS:', round((f / t), 1))


# Releasing video reader and writer
video.release()
writer.release()


"""
Some comments

What is a FOURCC?
    FOURCC is short for "four character code" - an identifier for a video codec,
    compression format, colour or pixel format used in media files.
    http://www.fourcc.org


Parameters for cv2.VideoWriter():
    filename - Name of the output video file.
    fourcc - 4-character code of codec used to compress the frames.
    fps	- Frame rate of the created video.
    frameSize - Size of the video frames.
    isColor	- If it True, the encoder will expect and encode colour frames.
"""

Frame number 1 took 3.11579 seconds
Frame number 2 took 0.87590 seconds
Frame number 3 took 0.86650 seconds
Frame number 4 took 0.85117 seconds
Frame number 5 took 0.82223 seconds
Frame number 6 took 0.87551 seconds
Frame number 7 took 1.30070 seconds
Frame number 8 took 1.30462 seconds
Frame number 9 took 1.24638 seconds
Frame number 10 took 0.83051 seconds
Frame number 11 took 1.08565 seconds
Frame number 12 took 0.93969 seconds
Frame number 13 took 0.85007 seconds
Frame number 14 took 0.84890 seconds
Frame number 15 took 0.87037 seconds
Frame number 16 took 0.87103 seconds
Frame number 17 took 0.83870 seconds
Frame number 18 took 0.91306 seconds
Frame number 19 took 0.84403 seconds
Frame number 20 took 1.23020 seconds
Frame number 21 took 1.25195 seconds
Frame number 22 took 1.23541 seconds
Frame number 23 took 0.88265 seconds
Frame number 24 took 0.84884 seconds
Frame number 25 took 0.82547 seconds
Frame number 26 took 0.82790 seconds
Frame number 27 took 0.84285 seconds
Frame numb

'\nSome comments\n\nWhat is a FOURCC?\n    FOURCC is short for "four character code" - an identifier for a video codec,\n    compression format, colour or pixel format used in media files.\n    http://www.fourcc.org\n\n\nParameters for cv2.VideoWriter():\n    filename - Name of the output video file.\n    fourcc - 4-character code of codec used to compress the frames.\n    fps\t- Frame rate of the created video.\n    frameSize - Size of the video frames.\n    isColor\t- If it True, the encoder will expect and encode colour frames.\n'